## Initial approach

In [7]:
from unittest.mock import MagicMock

class ProductionClass:
    ...

thing = ProductionClass()
thing.method = MagicMock(return_value=3)
thing.method(3, 4, 5, key='value')

thing.method.assert_called_with(3, 4, 5, key='value')

None


In [51]:
from jinja2.nativetypes import NativeEnvironment
import pandas as pd

TEMPLATE = """
Sample Report

{{title}}

{{description}}

{{data}}
"""

title = "Very Important Report"
description = "Sample description"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

env = NativeEnvironment()
template = env.from_string(TEMPLATE)
report = template.render(title=title, description=description, data=df.to_string())
print(report)



Sample Report

Very Important Report

Sample description

   col1  col2
0     1     3
1     2     4


## Let's use a function

In [20]:
from jinja2.nativetypes import NativeEnvironment
import pandas as pd

template_str = """
Sample Report

{{title}}

{{description}}

{{data}}
"""

title = "Very Important Report"
description = "Sample description"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

def generate_report(title, description, data):
    env = NativeEnvironment()
    template = env.from_string(template_str)
    return template.render(title=title, description=description, data=data)

from pprint import pprint
pprint(generate_report(title, description, df.to_string()))

('\n'
 'Sample Report\n'
 '\n'
 'Very Important Report\n'
 '\n'
 'Sample description\n'
 '\n'
 '   col1  col2\n'
 '0     1     3\n'
 '1     2     4')


In [21]:
title = "sample title"
description = "sample description"
data = "42"

report = generate_report(title, description, data)
print(report)
description in report


Sample Report

sample title

sample description

42


True

In [25]:
import unittest
from unittest import TestCase

class GenerateReport(TestCase):
    def test_success(self):
        title = "sample title"
        description = "sample description"
        data = "42"

        report = generate_report(title, description, data)
        self.assertTrue(title in report)
        self.assertTrue(description in report)
        self.assertTrue(data in report)

    def test_failure(self):
        generate_report(None, None, None)



unittest.main(argv=[''], verbosity=2, exit=False)

test_failure (__main__.GenerateReport) ... ok
test_success (__main__.GenerateReport) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.013s

OK


In [6]:
import unittest

class TestStringMethods(unittest.TestCase):  # A testcase is created by subclassing unittest.TestCase

    # The three individual tests are defined with methods whose names start with the
    # letters test. This naming convention informs the test runner about which methods
    # represent tests.
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

unittest.main(argv=[''], verbosity=2, exit=False)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


In [6]:
s = 'hello world'
s.split() == ['hello', 'world']

True

In [4]:
class TestStringMethodsWithPytest:

    # The three individual tests are defined with methods whose names start with the
    # letters test. This naming convention informs the test runner about which methods
    # represent tests.
    def test_upper(self):
        assert 'foo'.upper()=='FOO'

    def test_isupper(self):
        assert 'FOO'.isupper() is True
        assert 'Foo'.isupper() is False

    def test_split(self):
        s = 'hello world'
        assert s.split() == ['hello', 'world']
        # check that s.split fails when the separator is not a string
        #with self.assertRaises(TypeError):
        #    s.split(2)

import pytest
pytest.main(["-x", ".", "-vv"])

============================= test session starts ==============================
platform darwin -- Python 3.10.2, pytest-7.1.3, pluggy-1.0.0 -- /Users/jmgomes/Library/Caches/pypoetry/virtualenvs/python-knowledge-base-0_fYsG-I-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/jmgomes/Projects/RuntimeRevolution/python-knowledge-base
collecting ... collected 0 items

============================ no tests ran in 0.01s =============================


<ExitCode.NO_TESTS_COLLECTED: 5>

In [31]:
report = generate_report(None, None, None)
report.replace('\n', '').replace('None', '')

'Sample Report'

## Validations

In [36]:
from jinja2.nativetypes import NativeEnvironment
import pandas as pd

TEMPLATE = """
Sample Report

{{title}}

{{description}}

{{data}}
"""

title = "Very Important Report"
description = "Sample description"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

def generate_report(title, description, data):
    if title is None or description is None or data is None:
        return
    env = NativeEnvironment()
    template = env.from_string(TEMPLATE)
    return template.render(title=title, description=description, data=data)


generate_report(None, None, None) is None

True

In [38]:
import unittest
from unittest import TestCase

class GenerateReport(TestCase):
    def setUp(self):
        self.title = "sample title"
        self.description = "sample description"
        self.data = "42"

    def test_success(self):
        report = generate_report(self.title, self.description, self.data)
        self.assertTrue(self.title in report)
        self.assertTrue(self.description in report)
        self.assertTrue(self.data in report)

    def test_failure_1_params(self):
        self.assertIsNone(generate_report(self.title, None, self.data))

    def test_failure_3_params(self):
        self.assertIsNone(generate_report(None, None, None))



unittest.main(argv=[''], verbosity=2, exit=False)

test_failure_1_params (__main__.GenerateReport) ... ok
test_failure_3_params (__main__.GenerateReport) ... ok
test_success (__main__.GenerateReport) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.013s

OK


## Set responsibilities

In [3]:
import pandas as pd
from jinja2.nativetypes import NativeEnvironment
from dataclasses import dataclass

TEMPLATE = """
Report

{{title}}

{{description}}

{{data}}
"""

title = "Very Important Report"
description = "Sample description"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

@dataclass
class GenerateReport:
    title: str
    description: str
    data: str

    def __post_init__(self):
        self._validate_input()
        self._get_template()

    def _validate_input(self):
        if self.title is None or self.description is None or self.data is None:
            raise Exception("Parameters are missing")

    def _get_template(self):
        env = NativeEnvironment()
        self.template = env.from_string(TEMPLATE)

    def __call__(self, *args, **kwargs):
        return self.template.render(title=self.title, description=self.description, data=self.data)

generate_report = GenerateReport(title, description, df.to_string())
print(generate_report())


Report

Very Important Report

Sample description

   col1  col2
0     1     3
1     2     4


In [2]:
import unittest
from unittest import TestCase

class GenerateReportTest(TestCase):
    def setUp(self):
        self.title = "sample title"
        self.description = "sample description"
        self.data = "42"

    def test_success(self):
        report = GenerateReport(self.title, self.description, self.data)()
        self.assertTrue(self.title in report)
        self.assertTrue(self.description in report)
        self.assertTrue(self.data in report)

    def test_failure_1_params(self):
        report = GenerateReport(self.title, None, self.data)()
        self.assertIsNone(report)

    def test_failure_3_params(self):
        report = generate_report(None, None, None)()
        self.assertIsNone(report)



unittest.main(argv=[''], verbosity=2, exit=False)

test_failure_1_params (__main__.GenerateReportTest) ... ERROR
test_failure_3_params (__main__.GenerateReportTest) ... ERROR
test_success (__main__.GenerateReportTest) ... ERROR

ERROR: test_failure_1_params (__main__.GenerateReportTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/k1/hrvz78210fl_2fl85sb86f600000gn/T/ipykernel_50354/2146599581.py", line 17, in test_failure_1_params
    report = GenerateReport(self.title, None, self.data)()
NameError: name 'GenerateReport' is not defined

ERROR: test_failure_3_params (__main__.GenerateReportTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/k1/hrvz78210fl_2fl85sb86f600000gn/T/ipykernel_50354/2146599581.py", line 21, in test_failure_3_params
    report = generate_report(None, None, None)()
NameError: name 'generate_report' is not defined

ERROR: test_success (__main__.GenerateR

In [7]:
import unittest
from unittest import TestCase

class GenerateReportTest(TestCase):
    def setUp(self):
        self.title = "sample title"
        self.description = "sample description"
        self.data = "42"

    def test_success(self):
        report = GenerateReport(self.title, self.description, self.data)()
        self.assertTrue(self.title in report)
        self.assertTrue(self.description in report)
        self.assertTrue(self.data in report)

    def test_failure_1_params(self):
        with self.assertRaises(Exception) as cm:
            GenerateReport(self.title, None, self.data)()

        self.assertEqual(cm.exception.args[0], "Parameters are missing")

    def test_failure_3_params(self):
        with self.assertRaises(Exception) as cm:
            GenerateReport(None, None, None)()

        self.assertEqual(cm.exception.args[0], "Parameters are missing")



unittest.main(argv=[''], verbosity=2, exit=False)

test_failure_1_params (__main__.GenerateReportTest) ... ok
test_failure_3_params (__main__.GenerateReportTest) ... ok
test_success (__main__.GenerateReportTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


In [16]:
import pandas as pd
from jinja2.nativetypes import NativeEnvironment
from typing import ClassVar
from dataclasses import dataclass

TEMPLATE = """
Report

{{title}}

{{description}}

{{data}}
"""

TEMPLATE_OUTRO = """
OUTRO

{{title}}

{{description}}

{{data}}
"""

title = "Very Important Report"
description = "Sample description"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

data = {"title": title, "description": description, "data": df.to_string()}

@dataclass
class GenerateReport:
    data: dict
    TEMPLATE_VARIABLES: ClassVar[set] = {"title", "description", "data"}
    TEMPLATE_NAME: ClassVar[str] = TEMPLATE

    def __post_init__(self):
        self._validate_input()
        self._get_template()

    def _validate_input(self):
        missing = set(self.data.keys()) ^ self.TEMPLATE_VARIABLES
        if missing:
            raise Exception(f"The following keys are missing: {missing}")

    def _get_template(self):
        env = NativeEnvironment()
        self.template = env.from_string(self.TEMPLATE_NAME)

    def __call__(self, *args, **kwargs):
        return self.template.render(**self.data)

@dataclass
class GenRepo2(GenerateReport):
    TEMPLATE_NAME: ClassVar[str] = TEMPLATE_OUTRO

report = GenRepo2(data)()
print(report)


OUTRO 

Very Important Report

Sample description

   col1  col2
0     1     3
1     2     4


In [15]:
import unittest
from unittest import TestCase

class GenerateReportTest(TestCase):
    def setUp(self):
        self.title = "sample title"
        self.description = "sample description"
        self.data = "42"

        self.template_data = {"title": self.title,
        "description": self.description,
        "data": self.data}

    def test_success(self):
        report = GenerateReport(self.template_data)()
        self.assertTrue(self.title in report)
        self.assertTrue(self.description in report)
        self.assertTrue(self.data in report)

    def test_failure_1_params(self):
        self.template_data.pop('description')
        with self.assertRaises(Exception) as cm:
            GenerateReport(self.template_data)()

        self.assertEqual(cm.exception.args[0], "The following keys are missing: {'description'}")

    def test_failure_3_params(self):
        with self.assertRaises(Exception) as cm:
            GenerateReport({})()

        self.assertEqual(cm.exception.args[0], "The following keys are missing: {'title', 'description', 'data'}")



unittest.main(argv=[''], verbosity=2, exit=False)

test_failure_1_params (__main__.GenerateReportTest) ... ok
test_failure_3_params (__main__.GenerateReportTest) ... ok
test_success (__main__.GenerateReportTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


In [13]:
def tt():
    data = {'col1': [1, 2], 'col2': [3, 4]}
    missing = set(data.keys()) ^ {"col1", "col2"}
    if missing:
        return False
    return True

tt()

True

In [28]:
from dataclasses import dataclass
from jinja2 import PackageLoader, select_autoescape
from jinja2.environment import Environment
from jinja2 import meta

import pandas as pd


v = Environment(loader=PackageLoader("sample_django_project"),
    autoescape=select_autoescape())
template = v.get_template("report.html")


# df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
# data = {"latitude": 2, "longitude": 43, "columns": df.to_string()}

parsed_content = v.parse(template)
meta.find_undeclared_variables(parsed_content)
# from pprint import pprint
# pprint(template.render(**data))

set()